In [3]:
from torch.autograd import Variable
import torch.onnx
import torchvision

In [4]:
dummy_input = Variable(torch.randn(10, 3, 224, 224))
model = torchvision.models.vgg11(pretrained=True)

In [5]:
# Providing input and output names sets the display names for values
# within the model's graph. Setting these does not change the semantics
# of the graph; it is only for readability.
#
# The inputs to the network consist of the flat list of inputs (i.e.
# the values you would pass to the forward() method) followed by the
# flat list of parameters. You can partially specify names, i.e. provide
# a list here shorter than the number of inputs to the model, and we will
# only set that subset of names, starting from the beginning.
#input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
#output_names = [ "output1" ]

torch.onnx.export(model, dummy_input, "vgg.onnx", verbose=True)

graph(%0 : Float(10, 3, 224, 224)
      %1 : Float(64, 3, 3, 3)
      %2 : Float(64)
      %3 : Float(128, 64, 3, 3)
      %4 : Float(128)
      %5 : Float(256, 128, 3, 3)
      %6 : Float(256)
      %7 : Float(256, 256, 3, 3)
      %8 : Float(256)
      %9 : Float(512, 256, 3, 3)
      %10 : Float(512)
      %11 : Float(512, 512, 3, 3)
      %12 : Float(512)
      %13 : Float(512, 512, 3, 3)
      %14 : Float(512)
      %15 : Float(512, 512, 3, 3)
      %16 : Float(512)
      %17 : Float(4096, 25088)
      %18 : Float(4096)
      %19 : Float(4096, 4096)
      %20 : Float(4096)
      %21 : Float(1000, 4096)
      %22 : Float(1000)) {
  %23 : Float(10, 64, 224, 224) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[1, 1]](%0, %1, %2), scope: VGG/Sequential[features]/Conv2d[0]
  %24 : Float(10, 64, 224, 224) = onnx::Relu(%23), scope: VGG/Sequential[features]/ReLU[1]
  %25 : Float(10, 64, 112, 112) = onnx::MaxPool[kernel_shape=[2, 2], pads=[0, 0, 0, 

In [1]:
import onnx

# Load the ONNX model
model = onnx.load("vgg.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
#onnx.helper.printable_graph(model.graph)

In [7]:
model.graph.input

[name: "0"
type {
  tensor_type {
    elem_type: FLOAT
    shape {
      dim {
        dim_value: 10
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 224
      }
      dim {
        dim_value: 224
      }
    }
  }
}
, name: "1"
type {
  tensor_type {
    elem_type: FLOAT
    shape {
      dim {
        dim_value: 64
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 3
      }
    }
  }
}
, name: "2"
type {
  tensor_type {
    elem_type: FLOAT
    shape {
      dim {
        dim_value: 64
      }
    }
  }
}
, name: "3"
type {
  tensor_type {
    elem_type: FLOAT
    shape {
      dim {
        dim_value: 128
      }
      dim {
        dim_value: 64
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 3
      }
    }
  }
}
, name: "4"
type {
  tensor_type {
    elem_type: FLOAT
    shape {
      dim {
        dim_value: 128
      }
    }
  }
}
, name: "5"
typ

In [2]:
import caffe2.python.onnx.backend as backend
import numpy as np

rep = backend.prepare(model, device="CPU") # or "CPU"
# For the Caffe2 backend:
#     rep.predict_net is the Caffe2 protobuf for the network
#     rep.workspace is the Caffe2 workspace for the network
#       (see the class caffe2.python.onnx.backend.Workspace)
outputs = rep.run(np.random.randn(10, 3, 224, 224).astype(np.float32))
# To run networks with more than one input, pass a tuple
# rather than a single numpy ndarray.
print(outputs[0])

[[-2.589579   -0.05874371 -0.10375194 ... -1.4284072  -0.0739091
   2.4973583 ]
 [-1.8856354   0.24618089  0.42049494 ... -1.209184   -0.7466364
   3.5512092 ]
 [-2.155446   -0.05488757  0.38757893 ... -1.112158   -0.43493274
   2.7320828 ]
 ...
 [-2.3419056  -0.1237722  -0.23530596 ... -1.2911285  -0.4330385
   2.886316  ]
 [-1.9893287   0.33303982  0.11155938 ... -0.96714675 -0.35925993
   3.0011141 ]
 [-2.0039675   0.21027038  0.39200854 ... -1.1921556  -0.17904913
   2.7132797 ]]


In [12]:
import numpy
import onnx
from onnx import numpy_helper

# Preprocessing: create a Numpy array
numpy_array = numpy.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], dtype=float)
print('Original Numpy array:\n{}\n'.format(numpy_array))

# Convert the Numpy array to a TensorProto
tensor = numpy_helper.from_array(numpy_array)
print('TensorProto:\n{}'.format(tensor))

# Convert the TensorProto to a Numpy array
new_array = numpy_helper.to_array(tensor)
print('After round trip, Numpy array:\n{}'.format(numpy_array))
print(numpy_array.mean())
print()

# Save the TensorProto
with open('tensor.pb', 'wb') as f:
    f.write(tensor.SerializeToString())
print('tensor.SerializeToString()')
print(tensor.SerializeToString())
print(pickle.dumps([[1., 2., 3.],[4., 5., 6.]]))
print('end')

# Load a TensorProto
new_tensor = onnx.TensorProto()
with open('tensor.pb', 'rb') as f:
    new_tensor.ParseFromString(f.read())
print('After saving and loading, new TensorProto:\n{}'.format(new_tensor))

Original Numpy array:
[[1. 2. 3.]
 [4. 5. 6.]]

TensorProto:
dims: 2
dims: 3
data_type: DOUBLE
raw_data: "\000\000\000\000\000\000\360?\000\000\000\000\000\000\000@\000\000\000\000\000\000\010@\000\000\000\000\000\000\020@\000\000\000\000\000\000\024@\000\000\000\000\000\000\030@"

After round trip, Numpy array:
[[1. 2. 3.]
 [4. 5. 6.]]
3.5

tensor.SerializeToString()
b'\x08\x02\x08\x03\x10\x0bJ0\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x18@'
b'\x80\x03]q\x00(]q\x01(K\x01K\x02K\x03e]q\x02(K\x04K\x05K\x06ee.'
end
After saving and loading, new TensorProto:
dims: 2
dims: 3
data_type: DOUBLE
raw_data: "\000\000\000\000\000\000\360?\000\000\000\000\000\000\000@\000\000\000\000\000\000\010@\000\000\000\000\000\000\020@\000\000\000\000\000\000\024@\000\000\000\000\000\000\030@"



In [1]:
import onnx
from onnx import helper
from onnx import AttributeProto, TensorProto, GraphProto

In [3]:
AttributeProto

onnx.onnx_ONNX_REL_1_3_ml_pb2.AttributeProto

In [2]:
# The protobuf definition can be found here:
# https://github.com/onnx/onnx/blob/master/onnx/onnx.proto


# Create one input (ValueInfoProto)
X = helper.make_tensor_value_info('X', TensorProto.FLOAT, [1, 2])

# Create one output (ValueInfoProto)
Y = helper.make_tensor_value_info('Y', TensorProto.FLOAT, [1, 4])

# Create a node (NodeProto)
node_def = helper.make_node(
    'Pad', # node name
    ['X'], # inputs
    ['Y'], # outputs
    mode='constant', # Attributes
    value=1.5,
    pads=[0, 1, 0, 1],
)

# Create the graph (GraphProto)
graph_def = helper.make_graph(
    [node_def],
    "test-model",
    [X],
    [Y],
)

# Create the model (ModelProto)
model_def = helper.make_model(graph_def,
                              producer_name='onnx-example')

print('The ir_version in model: {}\n'.format(model_def.ir_version))
print('The producer_name in model: {}\n'.format(model_def.producer_name))
print('The graph in model:\n{}'.format(model_def.graph))
onnx.checker.check_model(model_def)
print('The model is checked!')

The ir_version in model: 3

The producer_name in model: onnx-example

The graph in model:
node {
  input: "X"
  output: "Y"
  op_type: "Pad"
  attribute {
    name: "mode"
    s: "constant"
    type: STRING
  }
  attribute {
    name: "pads"
    ints: 0
    ints: 1
    ints: 0
    ints: 1
    type: INTS
  }
  attribute {
    name: "value"
    f: 1.5
    type: FLOAT
  }
}
name: "test-model"
input {
  name: "X"
  type {
    tensor_type {
      elem_type: FLOAT
      shape {
        dim {
          dim_value: 1
        }
        dim {
          dim_value: 2
        }
      }
    }
  }
}
output {
  name: "Y"
  type {
    tensor_type {
      elem_type: FLOAT
      shape {
        dim {
          dim_value: 1
        }
        dim {
          dim_value: 4
        }
      }
    }
  }
}

The model is checked!
